In [ ]:
import pandas as pd

raw_url = "https://raw.githubusercontent.com/Parkss0/251119/4f5d8d350f3c777d33b38827471292245092756d/260210_duplicate_remove.csv"

# 인코딩 자동 대응(utf-8 / utf-8-sig / cp949)
for enc in ["utf-8", "utf-8-sig", "cp949", "euc-kr"]:
    try:
        df = pd.read_csv(raw_url, encoding=enc)
        print("SUCCESS encoding =", enc, "| shape =", df.shape)
        break
    except UnicodeDecodeError as e:
        print("FAIL encoding =", enc, "|", e)

df.head()


SUCCESS encoding = utf-8 | shape = (10380, 8)


,Unnamed: 0,std_smiles,canonical_smiles,max_phase,oral,parenteral,prodrug,max_phase_num
0,0,Brc1c(NC2=NCCN2)ccc2nccnc12,Brc1c(NC2=NCCN2)ccc2nccnc12,4,0,0,0,4
1,1,C#CC(O)(/C=C/Cl)CC,C#CC(O)(/C=C/Cl)CC,4,1,0,0,4
2,2,C#CC1(O)CC[C@H]2[C@@H]3CC=C4CC(=O)CCC4[C@H]3CC...,C#CC1(O)CC[C@H]2[C@@H]3CCC4=CC(=O)CC[C@@H]4[C@...,4,1,0,0,4
3,3,C#CC1(OC(N)=O)CCCCC1,C#CC1(OC(N)=O)CCCCC1,4,1,0,0,4
4,4,C#CCC(Cc1cnc2nc(N)nc(N)c2n1)c1ccc(C(=O)NC(CCC(...,C#CCC(Cc1cnc2nc(N)nc(N)c2n1)c1ccc(C(=O)N[C@@H]...,4,0,1,0,4


In [1]:
import pandas as pd

raw_url = "https://raw.githubusercontent.com/Parkss0/251119/c58afd605dd2f0553261daff74105dc8dbe35dcf/missing_xlogp3_std_smiles.csv"

df_missing = pd.read_csv(raw_url, encoding="utf-8-sig")
print(df_missing.shape)
display(df_missing.head())


(605, 1)


,std_smiles
0,C#C[C@]1(O)CC[C@H]2[C@@H]3CCC4=C/C(=N/O)CC[C@@...
1,C#C[C@]1(OC(C)=O)CC[C@H]2[C@@H]3CCC4=C/C(=N/O)...
2,C/C(=C\c1csc(C)n1)[C@@H]1C[C@@H]2O[C@]2(C)CCC[...
3,C/C(=N\O)c1ccc(OCC(=O)N2CCCCC2)cc1
4,C/C(O)=C(\C#N)C(=O)Nc1ccc(C(F)(F)F)cc1


In [ ]:
import time
import csv
import pandas as pd
import numpy as np
import requests

PUG = "https://pubchem.ncbi.nlm.nih.gov/rest/pug"

session = requests.Session()
session.headers.update({
    "User-Agent": "Mozilla/5.0 (colab; pubchem xlogp lookup)"
})

def fetch_xlogp_batch(smiles_batch, timeout=60, max_retries=3):
    """
    smiles_batch: list[str]
    return: dict {canonical_smiles: xlogp or None}
    """
    url = f"{PUG}/compound/smiles/property/XLogP,CanonicalSMILES/CSV"
    payload = {"smiles": "\n".join(smiles_batch)}  # ✅ 여러 SMILES를 줄바꿈으로 전달

    for attempt in range(max_retries):
        try:
            r = session.post(url, data=payload, timeout=timeout)
            if r.status_code == 200 and r.text.strip():
                reader = csv.DictReader(r.text.splitlines())
                out = {}
                for row in reader:
                    csmi = (row.get("CanonicalSMILES") or "").strip()
                    x = (row.get("XLogP") or "").strip()
                    if not csmi:
                        continue
                    try:
                        out[csmi] = float(x) if x != "" else None
                    except ValueError:
                        out[csmi] = None
                return out

            # 일시적 오류면 backoff
            if r.status_code in (429, 500, 503):
                time.sleep((attempt + 1) * 2)
                continue

            # 그 외(400 등)는 그냥 빈 dict
            return {}

        except requests.RequestException:
            time.sleep((attempt + 1) * 2)

    return {}

def fetch_xlogp_single(smiles, timeout=60, max_retries=3):
    """
    단건 fallback: {smiles: xlogp}, {canonical_smiles: xlogp} 둘 다 채워주기
    """
    url = f"{PUG}/compound/smiles/property/XLogP,CanonicalSMILES/JSON"
    for attempt in range(max_retries):
        try:
            r = session.post(url, data={"smiles": smiles}, timeout=timeout)
            if r.status_code == 200:
                js = r.json()
                props = js.get("PropertyTable", {}).get("Properties", [])
                if not props:
                    return {}
                x = props[0].get("XLogP", None)
                csmi = props[0].get("CanonicalSMILES", None)
                try:
                    x = float(x) if x is not None else None
                except ValueError:
                    x = None
                out = {smiles: x}
                if csmi:
                    out[str(csmi)] = x
                return out

            if r.status_code in (429, 500, 503):
                time.sleep((attempt + 1) * 2)
                continue
            return {}

        except Exception:
            time.sleep((attempt + 1) * 2)
    return {}

# =========================
# 실행 파트
# =========================
assert "std_smiles" in df.columns, "df에 std_smiles 컬럼이 없어!"

smiles_series = df["std_smiles"].dropna().astype(str)
unique_smiles = smiles_series.unique().tolist()
print("unique std_smiles:", len(unique_smiles))

# 1) 배치 조회
batch_size = 50          # 너무 크면 400 뜰 수 있어 20~80 사이 추천
sleep_sec = 0.25         # 초당 4회 정도(안전하게)
mapping = {}

for i in range(0, len(unique_smiles), batch_size):
    batch = unique_smiles[i:i + batch_size]
    got = fetch_xlogp_batch(batch)
    mapping.update(got)

    if (i // batch_size) % 10 == 0:
        print(f"{min(i+batch_size, len(unique_smiles))} / {len(unique_smiles)} batches processed, mapping size={len(mapping)}")

    time.sleep(sleep_sec)

# 2) 1차 매핑
df["xlogp3_pubchem"] = df["std_smiles"].astype(str).map(mapping)

# 3) 누락값 fallback(단건)
missing = df.loc[df["xlogp3_pubchem"].isna() & df["std_smiles"].notna(), "std_smiles"].astype(str).unique().tolist()
print("missing after batch:", len(missing))

for j, smi in enumerate(missing):
    mapping.update(fetch_xlogp_single(smi))
    if (j + 1) % 200 == 0:
        print(f"fallback {j+1} / {len(missing)}")
    time.sleep(sleep_sec)

# 4) 최종 매핑 + 저장
df["xlogp3_pubchem"] = df["std_smiles"].astype(str).map(mapping)

df.to_csv("df_with_xlogp3_pubchem.csv", index=False)
pd.DataFrame({"smiles_key": list(mapping.keys()), "xlogp3_pubchem": list(mapping.values())}) \
  .to_csv("xlogp3_mapping.csv", index=False)

print("Saved: df_with_xlogp3_pubchem.csv / xlogp3_mapping.csv")
print("xlogp filled rate:", df["xlogp3_pubchem"].notna().mean())


unique std_smiles: 10380
50 / 10380 batches processed, mapping size=0
550 / 10380 batches processed, mapping size=0
1050 / 10380 batches processed, mapping size=0
1550 / 10380 batches processed, mapping size=0
2050 / 10380 batches processed, mapping size=0
2550 / 10380 batches processed, mapping size=0
3050 / 10380 batches processed, mapping size=0
3550 / 10380 batches processed, mapping size=0
4050 / 10380 batches processed, mapping size=0
4550 / 10380 batches processed, mapping size=0
5050 / 10380 batches processed, mapping size=0
5550 / 10380 batches processed, mapping size=0
6050 / 10380 batches processed, mapping size=0
6550 / 10380 batches processed, mapping size=0
7050 / 10380 batches processed, mapping size=0
7550 / 10380 batches processed, mapping size=0
8050 / 10380 batches processed, mapping size=0
8550 / 10380 batches processed, mapping size=0
9050 / 10380 batches processed, mapping size=0
9550 / 10380 batches processed, mapping size=0
10050 / 10380 batches processed, mappi

In [2]:
# Colab: PubChem XLogP(=xlogp3) 최대 매칭 스크립트
import time, csv
import pandas as pd
import numpy as np
import requests

# =========================
# 0) 입력 CSV 읽기 (std_smiles 1개 컬럼)
# =========================
raw_url = "https://raw.githubusercontent.com/Parkss0/251119/c58afd605dd2f0553261daff74105dc8dbe35dcf/missing_xlogp3_std_smiles.csv"
df = pd.read_csv(raw_url, encoding="utf-8-sig")

# 컬럼명 통일 (혹시라도 다르면)
if "std_smiles" not in df.columns:
    df.columns = ["std_smiles"]

def clean_smiles(x):
    if pd.isna(x):
        return None
    s = str(x).strip()
    if s in ("", "nan", "NaN", "None"):
        return None
    return s

df["std_smiles"] = df["std_smiles"].map(clean_smiles)
df = df.dropna(subset=["std_smiles"]).drop_duplicates("std_smiles").reset_index(drop=True)
print("unique std_smiles:", len(df))

# =========================
# 1) PubChem 세션/유틸
# =========================
PUG = "https://pubchem.ncbi.nlm.nih.gov/rest/pug"

session = requests.Session()
session.headers.update({"User-Agent": "Mozilla/5.0 (colab; pubchem xlogp lookup)"})

def largest_fragment(smi: str) -> str:
    """염/mixture('.')이면 가장 긴 fragment만(대충 salt 제거 목적)"""
    if "." not in smi:
        return smi
    frags = [f for f in smi.split(".") if f.strip()]
    if not frags:
        return smi
    return max(frags, key=len)

def _sleep_backoff(attempt):
    time.sleep(min(10, 1.5 * (attempt + 1)))

def post_json(url, data=None, params=None, timeout=60, max_retries=4):
    for attempt in range(max_retries):
        try:
            r = session.post(url, data=data, params=params, timeout=timeout)
            if r.status_code == 200:
                return r.json()
            if r.status_code in (429, 500, 503):
                _sleep_backoff(attempt)
                continue
            return None
        except requests.RequestException:
            _sleep_backoff(attempt)
    return None

def post_csv(url, data=None, params=None, timeout=60, max_retries=4):
    for attempt in range(max_retries):
        try:
            r = session.post(url, data=data, params=params, timeout=timeout)
            if r.status_code == 200 and r.text.strip():
                return r.text
            if r.status_code in (429, 500, 503):
                _sleep_backoff(attempt)
                continue
            return None
        except requests.RequestException:
            _sleep_backoff(attempt)
    return None

# =========================
# 2) 조회 함수들
# =========================
def fetch_xlogp_exact(smiles: str):
    """
    exact: /compound/smiles/property/XLogP,CanonicalSMILES/JSON
    return: dict {xlogp, canonical, cid} or None
    """
    url = f"{PUG}/compound/smiles/property/XLogP,CanonicalSMILES/JSON"
    js = post_json(url, data={"smiles": smiles})
    if not js:
        return None

    props = js.get("PropertyTable", {}).get("Properties", [])
    if not props:
        return None

    # 여러 개면 XLogP 있는 것 우선
    best = None
    for p in props:
        x = p.get("XLogP", None)
        if x is None:
            continue
        best = p
        break
    if best is None:
        best = props[0]

    x = best.get("XLogP", None)
    try:
        x = float(x) if x is not None else None
    except Exception:
        x = None

    return {
        "xlogp": x,
        "canonical": best.get("CanonicalSMILES", None),
        "cid": best.get("CID", None),
    }

def fetch_xlogp_fastidentity(smiles: str, identity_type="same_connectivity"):
    """
    넓은 매칭: fastidentity + identity_type=same_connectivity
    - stereo/동위원소 무시하고 연결성 기준으로 CID 후보를 가져오는 느낌 :contentReference[oaicite:1]{index=1}
    return: dict {xlogp, canonical, cid} or None
    """
    # fastidentity에서 바로 XLogP까지 뽑아보는 시도(되면 1방)
    url = f"{PUG}/compound/fastidentity/smiles/property/XLogP,CanonicalSMILES/CSV"
    txt = post_csv(url, data={"smiles": smiles}, params={"identity_type": identity_type})
    if not txt:
        return None

    reader = csv.DictReader(txt.splitlines())
    rows = list(reader)
    if not rows:
        return None

    # 여러 후보 중 XLogP 있는 것 우선
    best = None
    for row in rows:
        x = (row.get("XLogP") or "").strip()
        if x != "":
            best = row
            break
    if best is None:
        best = rows[0]

    cid = (best.get("CID") or "").strip()
    csmi = (best.get("CanonicalSMILES") or "").strip()
    x = (best.get("XLogP") or "").strip()
    try:
        x = float(x) if x != "" else None
    except Exception:
        x = None

    return {"xlogp": x, "canonical": csmi if csmi else None, "cid": int(cid) if cid.isdigit() else None}

def fetch_xlogp_best(smiles: str):
    """
    최대로 찾아주는 파이프:
      1) exact(original)
      2) exact(largest fragment)
      3) fastidentity same_connectivity(original)
      4) fastidentity same_connectivity(largest fragment)
    """
    # 1) exact original
    r = fetch_xlogp_exact(smiles)
    if r and (r["xlogp"] is not None):
        return r, "exact"

    # 2) exact main fragment (salt/mixture 대비)
    main = largest_fragment(smiles)
    if main != smiles:
        r2 = fetch_xlogp_exact(main)
        if r2 and (r2["xlogp"] is not None):
            return r2, "exact_mainfrag"

    # 3) fastidentity original
    r3 = fetch_xlogp_fastidentity(smiles, identity_type="same_connectivity")
    if r3 and (r3["xlogp"] is not None):
        return r3, "fastidentity_same_connectivity"

    # 4) fastidentity main fragment
    if main != smiles:
        r4 = fetch_xlogp_fastidentity(main, identity_type="same_connectivity")
        if r4 and (r4["xlogp"] is not None):
            return r4, "fastidentity_mainfrag_same_connectivity"

    # 그래도 없으면 None
    return {"xlogp": None, "canonical": None, "cid": None}, "not_found"

# =========================
# 3) 실행
# =========================
try:
    from tqdm.auto import tqdm
except Exception:
    def tqdm(x, **kwargs): return x

sleep_sec = 0.20  # PubChem 예의상 너무 빠르게 때리지 않기(대략 5 req/sec 이하 권장)
results = []

for smi in tqdm(df["std_smiles"].tolist(), total=len(df), desc="PubChem XLogP lookup"):
    data, how = fetch_xlogp_best(smi)
    results.append({
        "std_smiles": smi,
        "xlogp3_pubchem": data["xlogp"],
        "matched_canonical_smiles": data["canonical"],
        "matched_cid": data["cid"],
        "match_mode": how,
    })
    time.sleep(sleep_sec)

res = pd.DataFrame(results)

print("found:", int(res["xlogp3_pubchem"].notna().sum()), "/", len(res))
print("still missing:", int(res["xlogp3_pubchem"].isna().sum()))

# 저장
res.to_csv("missing_xlogp3_std_smiles_with_xlogp3.csv", index=False, encoding="utf-8-sig")

still = res.loc[res["xlogp3_pubchem"].isna(), ["std_smiles"]].copy()
still.to_csv("still_missing_std_smiles.csv", index=False, encoding="utf-8-sig")

print("Saved:",
      "missing_xlogp3_std_smiles_with_xlogp3.csv",
      "/ still_missing_std_smiles.csv")


unique std_smiles: 605


PubChem XLogP lookup:   0%|          | 0/605 [00:00<?, ?it/s]

found: 557 / 605
still missing: 48
Saved: missing_xlogp3_std_smiles_with_xlogp3.csv / still_missing_std_smiles.csv
